# Data analysis of TMDB
Ih this task I am going to do some data analysis of movies and tv shows that are in The Movie DataBase.
The insights I am going to find is the following:
- What directors have made most movies and/or tv-shows.
- What actors have been the most active.
- What genres was the most popular per decade.

## Getting the data

I was at first going to just use [TMDB's API](https://developer.themoviedb.org/reference/intro/getting-started), but I encountered a problem. When I was trying to call for the 501th page (i.e. pages 500+) it gave me a error with the text `Invalid page: Pages start at 1 and max at 500. They are expected to be an integer.`, this is quite annoying since the amount of pages as today (16 01 2025) is a whoping `48184`. And with the page limit I would only be able to access **just a bit over 1%** using this method. I did then discover the [daily ID](https://developer.themoviedb.org/docs/daily-id-exports) lists that they have. This was pretty sweet, but sadly i stumbled upon a me problem this time, my code was not good enough and I did not know how to improve it further, a aproximatly 190 hours of processing time to find directors from just the `People` list? Not acceptable, I then went for something a bit easier, I found this lovely [.csv file](https://www.kaggle.com/datasets/alanvourch/tmdb-movies-daily-updates) that user Alan Vourc'h had uploaded to Kaggle, and that file is what I am goign to use for this project.

## Start of code
I am going to use Python as my coding language, since it is what I am the most comfortable with.

The very first thing I do is choose my modules for my code:
- I chose `pandas` as my module of choice to handle the .csv file.
- I chose `numpy` for its usefull math library.
- I chose `seaborn` for its posibility to visualize data in a friendly way.
- I chose `matplotlib.pyplot` for making graphing possible.
- I chose `ipywidgets` for making the graphs interactive.
- I chose `date` from `datetime` for removing movies that have not been released yet.
- I chose `Counter` from `collections` to go over how often a genre shows up.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from datetime import date
from collections import Counter

Then since the file itself has many columns that I do not need, I made a list for them, and put that list in to the `usecols` parametre in panas' `read_csv`.

In [2]:

fields = ["title","id","popularity","vote_average","vote_count","release_date","revenue","budget","runtime","genres","cast","director","writers","producers","imdb_rating","imdb_votes"] # Put the info you want here

df = pd.read_csv(r"Data\TMDB_all_movies.csv", skipinitialspace=True, usecols=fields)

Then, since there are some rows that had `NAN` as their values, i changed those to a 0 where it was fitting. (I did not do this for the movies with no credits, they are filtered out.) 
I also set the index to be the movies ID.

In [3]:
#Data cleaning START
df.set_index("id", inplace=True) #Sets the index to be the movies ID

for i in ["vote_average","vote_count","revenue","budget","imdb_rating","imdb_votes"]:
    df[i].fillna(0,inplace=True)
    #This replaces NAN values with in the list to 0


df["release_date"] = pd.to_datetime(df["release_date"]) #Turns the dates in to the date format

today = pd.to_datetime(date.today())
df = df[df["release_date"] <= today] #This removes movies that are in the dataset that are not out yet.

df.drop_duplicates(inplace = True)

df.dropna(inplace=True)
#Data cleaning END

C:\Users\Bruker\AppData\Local\Temp\ipykernel_16576\175845122.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[i].fillna(0,inplace=True)


## The First Question

So to find out who has directed the most movies, and how many movies they have directed I do a quick query on the `director` collumn.

In [ ]:
df["director"].describe(include="object")

count             226150
unique            106975
top       David DeCoteau
freq                 142
Name: director, dtype: object

And here I look at the `top` in the terminal to find out that it is David DeCoteau that has directed the most movies, and by looking at `freq` that he has directed 142 movies.

## The Second Question

Now to find the person whom has acted in the most movies, this is found in a very similar way to the first question, but now I will use the collumn `cast`.

In [5]:
df["cast"].describe(include="object")

count        226150
unique       223573
top       Mel Blanc
freq            139
Name: cast, dtype: object

And it is Mel Blanc that has played in the most movies, in 139 of them.

## The Third Question

Now to find out what genres are the most popular per decade, I first divide up each movie in to the decades that they are in.

In [6]:
df["decade"] = (df["release_date"].dt.year // 10) * 10

I then run a `df.head()` to make sure that the new collumn `decade` has been added.

In [7]:
df.head()

,title,vote_average,vote_count,release_date,revenue,runtime,budget,popularity,genres,cast,director,writers,producers,imdb_rating,imdb_votes,decade
id,,,,,,,,,,,,,,,,
2,Ariel,7.100,338.0,1988-10-21,0.0,73.0,0.0,12.553,"Comedy, Drama, Romance, Crime","Jyrki Olsonen, Jaakko Talaskivi, Olli Varja, H...",Aki Kaurismäki,Aki Kaurismäki,Aki Kaurismäki,7.4,8892.0,1980
3,Shadows in Paradise,7.300,402.0,1986-10-17,0.0,74.0,0.0,16.411,"Comedy, Drama, Romance","Sirkka Silin, Helmeri Pellonpää, Tanja Talaski...",Aki Kaurismäki,Aki Kaurismäki,Mika Kaurismäki,7.5,7681.0,1980
5,Four Rooms,5.900,2650.0,1995-12-09,4257354.0,98.0,4000000.0,19.476,Comedy,"Lili Taylor, Valeria Golino, Marisa Tomei, Mar...","Quentin Tarantino, Robert Rodriguez, Allison A...","Quentin Tarantino, Robert Rodriguez, Allison A...","Quentin Tarantino, Lawrence Bender, Alexandre ...",6.7,113177.0,1990
6,Judgment Night,6.500,333.0,1993-10-15,12136938.0,109.0,21000000.0,12.110,"Action, Crime, Thriller","Stephen Dorff, Everlast, Will Zahrn, Emilio Es...",Stephen Hopkins,"Jere Cunningham, Lewis Colick","Gene Levy, Marilyn Vance, Lloyd Segan",6.6,19493.0,1990
9,Sunday in August,7.135,26.0,2004-09-02,0.0,15.0,0.0,2.729,Drama,"Rita Lengyel, Milton Welsh","Marc Meyer, Anna Haas",Marc Meyer,Marc Meyer,6.8,14.0,2000


And it has!
I will now gothrough each movie and find its genres, I am going to test a few diffrent aproaches for this:
1. I will tally up the genres for each movie, so if a movie is a `drama` & `comedy` movie that was released in the decade `1980`, the decade `1980` will have `1 drama movie` "point" and `1 comedy movie` "point"
2. I will do similar as in 1., but instead each movie will count as a fraction (`1/n` where `n` is the amount of genres that movie has). So the same movie in method 1. the decade `1980` will have `0.5 drama movie` "points" and `0.5 comedy movie` "points".
3. Similar as to 1., but this time the `popularity` collumn will play in, if the movie in method 1. has a `popularity` score of `10` the decade `1980` will have `10 drama movie` "points" and `10 comedy movie` "points".
4. Similar as to 3, but this time based on method 2., so it will divide the popularity on the amount of genres. So then the decade `1980` will have `5 drama movie` "points" and `5 comedy movie` "points".

I wanted to do this with multiple different aproaches, the first and third might give a slightly "bloated" feeling when it comes to genres, since a movie rarely has only one genre. Therfore I made the second and fourth aproaches. And for the first and second aproach versus third and fourth aproach, I thought it would be interesting to see if the most common genres per dacade was the most popular ones

I start with finding out all the different decades in the data set.

In [8]:
print(df["decade"].unique())

[1980 1990 2000 1970 1940 1920 1960 1930 1950 2010 1910 1900 2020 1890]


Now with the diffrent decades, i go through each movie in the list, starting with the earliest decade. I then use the `pandas.explode()` to get a copy of each genre per movie by itself. Then I use `Counter` to count how many induvidual times each genre shows up, add it to a dictionary, and then go on to the next decade. I repeat this untuil there are no more decades.

In [9]:
def count_genres_once_per_movie() -> dict:
    """
    Goes through each movie in the data frame, going decade by decade, starting with the earliest decade.
    Then it splits up each genre in to its own "copy" of the movie with with `.explode()`.
    It then with `Counter()` it counts all the genres and how many times they showed up in that decade.
    Then it adds the value for each genre, the value is the amount of times a movie has that genre.
    Then it goes to the next decade.
    returns:
        dict: a dict of each decade, and its popularity
    """
    
    genre_popularity_per_decade = {}
    
    for decade in sorted(df["decade"].unique()):
        temporary_df = df[df["decade"] == decade]
        #I first use the pandas .explode() method to get each genre for itself
        all_genres_that_decade = temporary_df["genres"].str.split(", ").explode()
        genre_count = Counter(all_genres_that_decade)
        sorted_genres = dict(sorted(genre_count.items(), key=lambda x: x[1], reverse=True)) #Makes it so that each genre in the decade dicts are sorted with the most often at the top.

        genre_popularity_per_decade[int(decade)] = dict(sorted_genres)

        
    return genre_popularity_per_decade    

def count_genres_weighted_by_count() -> dict:
    """Goes through each movie in the data frame, then for each movie, it gets its genres, and logs how many genres that movie id had.
    Then it splits up each genre in to its own "copy" of the movie with with `.explode()`.
    It then with `Counter()` it counts all the genres and how many times they showed up in that decade.
    Then it adds the value for each genre, the value is the amount of times a movie has that genre, divided on how many genres that movie had.
    Then it goes to the next decade.

    returns:
        dict: a dict of each decade, and its popularity
    """
    
    genre_popularity_per_decade = {}
    
    for decade in sorted(df["decade"].unique()):
        temporary_df = df[df["decade"] == decade]
        
        genre_weights = Counter()
        
        for genres in temporary_df["genres"]:
            genre_list = genres.split(", ")
            weight = 1 / len(genre_list)
            
            
            for genre in genre_list:
                genre_weights[genre] += weight
            
        rounded_weights = {genre: round(weight, 2) for genre, weight in genre_weights.items()} #Makes it so that each genre in the finished dict is rounded, if i rounded it in the loop above it did not work.
        sorted_genres = dict(sorted(rounded_weights.items(), key=lambda x: x[1], reverse=True)) #Makes it so that each genre in the decade dicts are sorted with the most often at the top.
            
        genre_popularity_per_decade[int(decade)] = dict(sorted_genres)
    
    return genre_popularity_per_decade  

def count_genres_weighted_by_popularity() -> dict:
    """
    Goes through each movie in the data frame, going decade by decade, starting with the earliest decade, also gets the movies popularity.
    Then it splits up each genre in to its own "copy" of the movie with with `.explode()`.
    It then with `Counter()` it counts all the genres and how many times they showed up in that decade, multiplied with that movies popularity.
    Then it goes to the next decade.
    
    returns:
        dict: a dict of each decade, and its popularity
    """
    
    genre_popularity_per_decade = {}
    
    for decade in sorted(df["decade"].unique()):
        temporary_df = df[df["decade"] == decade]
        
        genre_weights = Counter()
        
        for _, row in temporary_df.iterrows():
            genres = row["genres"].split(", ")
            popularity = row["popularity"]  # Get the popularity for the movie
            
            
            for genre in genres:
                genre_weights[genre] += popularity
            
        rounded_weights = {genre: round(weight, 2) for genre, weight in genre_weights.items()} #Makes it so that each genre in the finished dict is rounded, if i rounded it in the loop above it did not work.
        sorted_genres = dict(sorted(rounded_weights.items(), key=lambda x: x[1], reverse=True)) #Makes it so that each genre in the decade dicts are sorted with the most often at the top.
            
        genre_popularity_per_decade[int(decade)] = dict(sorted_genres)
    
    return genre_popularity_per_decade

def count_genres_weighted_by_popularity_and_count() -> dict:
    """
    Goes through each movie in the data frame, going decade by decade, starting with the earliest decade, also gets the movies popularity.
    Then it splits up each genre in to its own "copy" of the movie with with `.explode()`.
    It then with `Counter()` it counts all the genres and how many times they showed up in that decade, multiplied with that movies popularity, then divided by the amount of genres that movie had.
    Then it goes to the next decade.
    
    returns:
        dict: a dict of each decade, and its popularity
    """
    genre_popularity_per_decade = {}
    
    for decade in sorted(df["decade"].unique()):
        temporary_df = df[df["decade"] == decade]
        
        genre_weights = Counter()
        
        for _, row in temporary_df.iterrows():
            genres = row["genres"].split(", ")
            popularity = row["popularity"]  # Get the popularity for the movie
            weight = 1 / len(genres)
            
            
            for genre in genres:
                genre_weights[genre] += popularity*weight
            
        rounded_weights = {genre: round(weight, 2) for genre, weight in genre_weights.items()} #Makes it so that each genre in the finished dict is rounded, if i rounded it in the loop above it did not work.
        sorted_genres = dict(sorted(rounded_weights.items(), key=lambda x: x[1], reverse=True)) #Makes it so that each genre in the decade dicts are sorted with the most often at the top.
            
        genre_popularity_per_decade[int(decade)] = dict(sorted_genres)
    
    return genre_popularity_per_decade


I am also going to make a simple function to get the top `N` movie genres per decade.

In [10]:
def get_top_genres(genre_popularity: dict, top_n: int = 3) -> dict:
    """gets the top N genres for all the decades in the dict.

    Args:
        genre_popularity (dict): Use `first_way()`, `second_way()`, `third_way()` or `fourth_way()`
        top_n (int, optional): How many genres to look for. Defaults to 3.

    Returns:
        dict: the `genre_popularity` with the `top_n` genres as a dict
    """
    
    top_genres = {}
    
    for decade, genre_counts in genre_popularity.items():
        top_n_genres = dict(sorted(genre_counts.items(), key=lambda x: x[1], reverse=True)[:top_n])
        top_genres[decade] = top_n_genres

    return top_genres

So, time to see what the diffrent methodes and see what information they give us

In [54]:
decades = [str(i) for i in df["decade"].unique()]
decades = sorted(decades)

decade_tup = [(item,index + 1) for index, item in enumerate(decades)]

decade_picker = widgets.Dropdown(
    options=decade_tup,
    description="Decade:",
)


In [60]:
decades = [str(i) for i in df["decade"].unique()]
decades = sorted(decades)

decade_tup = [(item,index + 1) for index, item in enumerate(decades)]

decade_picker

df_genres = pd.DataFrame.from_dict(count_genres_once_per_movie(), orient="index")
df_genres.fillna(0,inplace=True)



def update_bar_plot(decade_value):

    plt.figure(figsize=(10, 6))
    plt.barh(df_genres.columns, df_genres.iloc[decade_value - 1])  # Adjust based on your data
    plt.title(f"Genre Distribution for the {decades[decade_value - 1]}")
    plt.xlabel("Count")
    plt.ylabel("Genres")
    plt.show()

# Use widgets.interactive to automatically refresh the plot when a new decade is selected
widgets.interactive(update_bar_plot, decade_value=decade_picker)

interactive(children=(Dropdown(description='Decade:', options=(('1890', 1), ('1900', 2), ('1910', 3), ('1920',…

In [12]:
df_genres.iloc[0]

Comedy             5.0
Fantasy            4.0
Horror             3.0
Drama              1.0
Family             1.0
Romance            1.0
History            0.0
Adventure          0.0
Crime              0.0
Science Fiction    0.0
Action             0.0
Western            0.0
War                0.0
Mystery            0.0
Thriller           0.0
Animation          0.0
Documentary        0.0
Music              0.0
TV Movie           0.0
Name: 1890, dtype: float64

In [13]:
def get_genre_moods(decade_id:int) -> list[float,float,float]:
    """Goes through each genre for that year, summing up the diffrent genre points in to one of thre categories, `positive`, `negative` and `neutral`. And then outputs value of those diffrent categories.

    Args:
        decade_id (int): The year to find the genre moods for. `decade_id[0] = 1890` ... `decade_id[13] = 2020´

    Returns:
        list[float,float,float]: The diffrent mood values, ´[positive, negative, neutral]´
    """

    positive_genres = ["Music", "Adventure", "Romance", "Family", "Comedy"]
    negative_genres = ["Thriller", "War", "Crime", "Horror"]
    neutral_genres = ["TV Movie", "Documentary", "Animation", "Mystery", "Western", "Action", "Science Fiction", "History", "Drama", "Fantasy"]
        
    positive_points = 0
    negative_points = 0
    neutral_points = 0
    
    decade = df_genres.iloc[decade_id]

    positive_points += sum(decade[mood] for mood in positive_genres)
    negative_points += sum(decade[mood] for mood in negative_genres)
    neutral_points += sum(decade[mood] for mood in neutral_genres)

    return [positive_points.item(),negative_points.item(),neutral_points.item()] #the `.item()` turns it in to a `float` from a `np.float64``


In [14]:
mood_data = {
    "Decade": [int(item) for item in decades], #Turns each decade in to a int
    "Positive": [get_genre_moods(i)[0] for i in range(len(decades))],
    "Negative": [get_genre_moods(i)[1] for i in range(len(decades))],
    "Neutral": [get_genre_moods(i)[2] for i in range(len(decades))]
}

In [15]:
df_mood = pd.DataFrame(mood_data)
#df_mood = df_mood.melt(id_vars="Decade", var_name="Mood", value_name="Value") #Melting causes each "shared data" to be split up.
df_mood

,Decade,Positive,Negative,Neutral
0,1890,7.0,3.0,5.0
1,1900,24.0,13.0,60.0
2,1910,448.0,135.0,859.0
3,1920,1190.0,284.0,1474.0
4,1930,3935.0,1046.0,3781.0
5,1940,3895.0,1639.0,4513.0
6,1950,5224.0,2393.0,6933.0
7,1960,6162.0,3032.0,8878.0
8,1970,6775.0,4537.0,13867.0
9,1980,8194.0,5052.0,16049.0


In [16]:
# Define the update function for the plot
def update_pie_plot(decade_value):
    decade_id = decade_value - 1
    colors = ["green", "red", "grey"]
    plt.figure(figsize=(10, 6))
    plt.title(f"Genre Distribution for the {decades[decade_id]}")
    plt.pie([df_mood["Positive"].iloc[decade_id], df_mood["Negative"].iloc[decade_id], df_mood["Neutral"].iloc[decade_id]] , labels=["Positive","Negative","Neutral"], autopct='%.0f%%', colors=colors) 
    plt.show()

# Use widgets.interactive to automatically refresh the plot when a new decade is selected
widgets.interactive(update_pie_plot, decade_value=decade_picker)

interactive(children=(Dropdown(description='Decade:', options=(('1890', 1), ('1900', 2), ('1910', 3), ('1920',…

Todo: Make a grap based on popularity of a genre / amount of movies in that genre

In [25]:
print(count_genres_once_per_movie())

{1890: {'Comedy': 5, 'Fantasy': 4, 'Horror': 3, 'Drama': 1, 'Family': 1, 'Romance': 1}, 1900: {'Drama': 26, 'Fantasy': 18, 'Comedy': 15, 'History': 8, 'Adventure': 6, 'Horror': 6, 'Crime': 5, 'Science Fiction': 3, 'Action': 3, 'Family': 2, 'Western': 1, 'War': 1, 'Romance': 1, 'Mystery': 1, 'Thriller': 1}, 1910: {'Drama': 571, 'Comedy': 291, 'Western': 141, 'Romance': 91, 'Adventure': 58, 'Crime': 50, 'History': 44, 'Horror': 39, 'War': 34, 'Fantasy': 28, 'Action': 23, 'Science Fiction': 19, 'Mystery': 18, 'Thriller': 12, 'Family': 8, 'Animation': 8, 'Documentary': 7}, 1920: {'Drama': 1056, 'Comedy': 644, 'Romance': 362, 'Adventure': 134, 'Crime': 121, 'Western': 119, 'Action': 86, 'Horror': 74, 'History': 66, 'War': 51, 'Mystery': 50, 'Fantasy': 49, 'Thriller': 38, 'Music': 31, 'Family': 19, 'Science Fiction': 16, 'Documentary': 16, 'Animation': 16}, 1930: {'Drama': 2149, 'Comedy': 1857, 'Romance': 1122, 'Crime': 644, 'Music': 521, 'Western': 443, 'Adventure': 335, 'Mystery': 335, 'Ac

In [18]:
print(count_genres_weighted_by_popularity()[1890])

{'Fantasy': 23.6, 'Horror': 21.08, 'Comedy': 15.92, 'Drama': 8.54, 'Family': 8.54, 'Romance': 8.54}


In [19]:
popularity_weighted = count_genres_weighted_by_popularity()
genre_counts = count_genres_weighted_by_count()

def genre_popularity() -> dict:
    """Generates a dict that is the popularity per genre divided by the amount of times that genre appeared in the dataset.

    Returns:
        dict: dictionary for each genre with its popularity avrage rating.
    """
    
    for decade in sorted(df["decade"].unique()):
        for genre in popularity_weighted[decade]:
            print(genre)

In [36]:
def genre_popularity() -> dict:
    """Generates a dict that is the popularity per genre divided by the amount of times that genre appeared in the dataset.

    Returns:
        dict: dictionary for each genre with its popularity avrage rating.
    """
    decade_dict = {}
    genre_dict = {}
    popularity = 0
    count = 0
    for decade in sorted(df["decade"].unique()):
        genre_dict = {}
        for genre in popularity_weighted[decade]:
            
            popularity = popularity_weighted[decade][genre]
            count = genre_counts[decade][genre]
            
            genre_dict.update({genre:round(popularity/count,2)})
        decade_dict.update({int(decade):genre_dict})
    return decade_dict

print(genre_popularity())

{1890: {'Fantasy': 8.58, 'Horror': 14.05, 'Comedy': 3.98, 'Drama': 34.16, 'Family': 34.16, 'Romance': 34.16}, 1900: {'Fantasy': 6.09, 'Drama': 2.79, 'Adventure': 14.44, 'Comedy': 2.57, 'Science Fiction': 25.71, 'History': 4.17, 'Crime': 8.58, 'Horror': 5.62, 'Action': 14.48, 'Family': 10.77, 'Western': 33.88, 'War': 20.64, 'Thriller': 12.97, 'Mystery': 1.82, 'Romance': 1.39}, 1910: {'Drama': 1.68, 'Comedy': 2.87, 'Romance': 3.97, 'Western': 1.25, 'Adventure': 3.54, 'History': 4.79, 'War': 5.21, 'Horror': 3.57, 'Fantasy': 4.98, 'Action': 4.71, 'Crime': 2.2, 'Science Fiction': 5.68, 'Thriller': 4.05, 'Animation': 5.01, 'Family': 6.47, 'Mystery': 1.72, 'Documentary': 1.68}, 1920: {'Drama': 3.15, 'Comedy': 2.94, 'Romance': 6.25, 'Adventure': 5.54, 'Horror': 10.88, 'Crime': 5.0, 'Fantasy': 8.88, 'Western': 2.37, 'Action': 5.61, 'History': 5.77, 'War': 7.57, 'Thriller': 9.74, 'Mystery': 4.52, 'Family': 10.85, 'Music': 4.72, 'Science Fiction': 9.39, 'Documentary': 5.64, 'Animation': 4.44}, 19

In [41]:
df_genre_popularity = pd.DataFrame.from_dict(genre_popularity(), orient="index")
df_genre_popularity.fillna(0,inplace=True)


decades = [str(i) for i in df["decade"].unique()]
decades = sorted(decades)

decade_tup = [(item,index + 1) for index, item in enumerate(decades)]

decade_picker = widgets.Dropdown(
    options=decade_tup,
    description="Decade:",
)
decade_picker


# Define the update function for the plot
def update_bar_plot_genre_popularity(decade_value):
    # Filter the data based on the selected decade
    df_decade = df[df["decade"] == decade_value]
    
    # Assuming 'df_genres' has columns for genres and rows for counts or other metrics
    # Adjust this as per your specific data structure
    plt.figure(figsize=(10, 6))
    plt.barh(df_genre_popularity.columns, df_genre_popularity.iloc[decade_value - 1])  # Adjust based on your data
    plt.title(f"Genre Popularity for the {decades[decade_value - 1]}")
    plt.xlabel("Count")
    plt.ylabel("Genres")
    plt.show()

# Use widgets.interactive to automatically refresh the plot when a new decade is selected
widgets.interactive(update_bar_plot_genre_popularity, decade_value=decade_picker)


interactive(children=(Dropdown(description='Decade:', options=(('1890', 1), ('1900', 2), ('1910', 3), ('1920',…

In [38]:
df_genre_popularity.head()

,Fantasy,Horror,Comedy,Drama,Family,Romance,Adventure,Science Fiction,History,Crime,Action,Western,War,Thriller,Mystery,Animation,Documentary,Music,TV Movie
1890,8.58,14.05,3.98,34.16,34.16,34.16,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1900,6.09,5.62,2.57,2.79,10.77,1.39,14.44,25.71,4.17,8.58,14.48,33.88,20.64,12.97,1.82,0.00,0.00,0.00,0.00
1910,4.98,3.57,2.87,1.68,6.47,3.97,3.54,5.68,4.79,2.20,4.71,1.25,5.21,4.05,1.72,5.01,1.68,0.00,0.00
1920,8.88,10.88,2.94,3.15,10.85,6.25,5.54,9.39,5.77,5.00,5.61,2.37,7.57,9.74,4.52,4.44,5.64,4.72,0.00
1930,19.87,13.34,3.82,4.54,13.62,7.96,8.51,13.08,7.81,6.16,7.53,3.52,10.52,10.00,7.00,7.95,2.67,4.64,1.78
